# Creating the Dash App

In [ ]:
#!pip install jupyter_dash dash dash-bootstrap-components

In [2]:
# Update path to allow for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.dash_app.generate_historical_viz import generate_historical_viz

In [3]:
#%%writefile ../app/visualization_app.py
import pandas as pd
from dash import Dash, dcc, html, Input, Output, ctx
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets

# Make sure to change this before deploying
df = pd.read_pickle("../data/processed/historical_outbreaks_df.pkl")

# Create a JupyterDash app and use bootstrap styling
app = JupyterDash(__name__, suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.COSMO])

# Define styles for the sidebar, content, and welcome sections
sidebar_style = { # these styles are modified from a template found here: https://dash-bootstrap-components.opensource.faculty.ai/examples/simple-sidebar/
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

content_style = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

welcome_style = {
    "text-align": "center",
    "margin-top": "16rem",
}

# Write the text content for the welcome section including hyperlinks
welcome_text_1 = """
Welcome to our Dash app! We're Team 23. Here you'll find the visualizations
we created for our capstone project that accompany our
"""
report_link = html.A(
    "final report",
    href="https://docs.google.com/document/d/1bIR_jaLaYRx6avVsLmQFBOuRI_6dxSKENzd0DeKN0oE/edit",
    target="_blank")

welcome_text_2 = """
. We used Demographic and
Health Surveys data to train a model that predicts the likelihood of an outbreak of a zoonotic
disease for a given country. We also mapped the history of major outbreaks and created visualizations of our 
findings that you can explore by navigating the menu on the left.
For more information about our project, check out our GitHub repository here:
"""

github_link = html.A(
    "https://github.com/alevine5/MADS-Capstone",
    href="https://github.com/alevine5/MADS-Capstone",
    target="_blank")

# Create the sidebar layout with navigation links
sidebar = html.Div(
    [
        html.H2("Menu", className="display-5"),
        html.Hr(),
        dbc.Nav(
            [
                dbc.NavLink("Welcome", href="/", active="exact"),
                dbc.NavLink("Risk Score Analysis", href="/page-1", active="exact"),
                dbc.NavLink("Historical Outbreaks", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=sidebar_style,
)

# Create the content layout
content = html.Div(
    style=content_style,
    id="page-content",
    children=[
        dcc.Loading(dcc.Graph(id="graph"), type="cube"),
    ]
)

# Create the welcome section
welcome = html.Div(style=welcome_style, children=[
    html.H1("Predicting Zoonotic Disease Outbreaks"),
    html.H2("By Alex Levine and Samuel Buxton"),
    html.P([welcome_text_1, report_link, welcome_text_2, github_link]),
    html.Div([
        html.A("Risk Score Analysis", href="/page-1", style={'margin-right': '20px'}),
        html.A("Historical Outbreaks", href="/page-2")
    ])
])

# For now use a placeholder for page 1
page_1 = html.Div([
    html.H4("Visual coming soon!"),
])

# Create the layout for page 2 with historical outbreaks visualization options
page_2 = html.Div([
    html.H4("Historical Outbreaks"),
    html.P("Select year range:"),
    dcc.RangeSlider(
        id="year-slider",
        marks={i: str(i) for i in range(-1800, 2011, 200)},
        min=-1800,
        max=2010,
        step=1,
        value=[1800, 2010],
    ),
    
    html.Div([
        dcc.Dropdown(
            id="country-dropdown",
            options=[
                {'label': 'Country Name', 'value': 'Name'},
                {'label': 'ISO-3', 'value': 'ISO-3'}
            ],
            value='Name',
            placeholder='Country Name',
            style={'width': '120px', 'display': 'inline-block'}
        ),
    ], style={'display': 'inline-block', 'margin-right': '20px', 'margin-top':'20px'}),
    
    html.Div([
        dcc.Dropdown(
            id="speed-dropdown",
            options=[
                {'label': '.25x', 'value': 125},
                {'label': '.5x', 'value': 250},
                {'label': '.75x', 'value': 375},
                {'label': '1x', 'value': 500},
                {'label': '1.5x', 'value': 750},
                {'label': '2x', 'value': 1000},
                {'label': '3x', 'value': 1500},
                {'label': '5x', 'value': 2500}
            ],
            value=500, 
            placeholder="1x",
            style={'width': '50px', 'display': 'inline-block'}
        ),
    ], style={'display': 'inline-block', 'margin-right': '20px'}),
    
    html.Div([
        dcc.Dropdown(
            id="pastyears-dropdown",
            options=[
                {'label': '0', 'value': 0},
                {'label': '10', 'value': 10},
                {'label': '20', 'value': 20},
                {'label': '30', 'value': 30},
                {'label': '40', 'value': 40},
                {'label': 'Past Years: 50', 'value': 50},
            ],
            value=50, 
            placeholder="Past Years",
            style={'width': '120px', 'display': 'inline-block'}
        ),
    ], style={'display': 'inline-block', 'margin-right': '20px'}),
    
    html.Div([
        html.Button(
            'Generate',
            id='generate-button',
            n_clicks=0,
            style={'display': 'inline-block'}
        ),
    ], style={'align-items': 'center'}),
    
    dcc.Loading(dcc.Graph(id="graph"), type="cube"),
])

# Define the overall layout of the app
app.layout = html.Div([dcc.Location(id="url"), sidebar, content])

# Callback to dynamically update the content based on the selected page
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def select_page(pathname):
    if pathname == "/":
        return welcome
    elif pathname == "/page-1":
        return page_1
    elif pathname == "/page-2":
        return page_2
    
# Callback to update the historical outbreaks graph based on user inputs
@app.callback(
    Output("graph", "figure"), 
    Input("year-slider", "value"),
    Input("country-dropdown", "value"),
    Input("speed-dropdown", "value"),
    Input("pastyears-dropdown", "value"),
    Input("generate-button", "n_clicks"),
)
def update_graph(years, name, duration, past, clicks):
    if ctx.triggered_id and "generate-button" != ctx.triggered_id:
        raise PreventUpdate
    return generate_historical_viz(
        df,
        start=years[0],
        end=years[1],
        country_name=name,
        duration=duration,
        past_years=past
    )

# Run the app
if __name__ == "__main__":
    app.run_server(mode='external')

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
